In [11]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt


In [16]:
def F_nu(t_p,nu_p,F_nup):
    m = 0.88 #using stAandard model
    p = 3.0 #using standard model
    a = (2*m)+0.5
    b = (p+5-(6*m))/2
    t=t_p
    nu=nu_p

    F_nu = F_nup*1.582*(t/t_p)**a*(nu/nu_p)**(5/2)*(1-np.exp(-(t/t_p)**(-(a+b))*(nu/nu_p)**(-(p+4)/2))
                                                
    print F_nu(1, 8,  0.0272)
                                                    
    return F_nu(1224.288326 , 8,  0.0272)                                           

SyntaxError: invalid syntax (<ipython-input-16-f5fd623be359>, line 11)